In [3]:
import pandas as pd
import csv

samsung_path = '005930_삼성전자.csv'
skhynix_path = '000660_SK하이닉스.csv'

samsung = pd.read_csv(samsung_path)
skhynix = pd.read_csv(skhynix_path)

df_samsung = pd.DataFrame(samsung)
df_skhynix = pd.DataFrame(skhynix)

print(df_samsung)
print(df_skhynix)

      code  name      date   open   high    low  close    volume  diffratio  \
0     5930  삼성전자  20150102  26800  26800  26540  26600   8774950   0.002261   
1     5930  삼성전자  20150105  26720  26720  26260  26660  10139500   0.002256   
2     5930  삼성전자  20150106  26300  26340  25760  25900  15235500  -0.028507   
3     5930  삼성전자  20150107  25880  26220  25640  26140  14322750   0.009266   
4     5930  삼성전자  20150108  26780  26780  26200  26280  14477600   0.005356   
...    ...   ...       ...    ...    ...    ...    ...       ...        ...   
2207  5930  삼성전자  20231221  74600  75000  74300  75000  13478766   0.002674   
2208  5930  삼성전자  20231222  75800  76300  75400  75900  14515608   0.012000   
2209  5930  삼성전자  20231226  76100  76700  75700  76600  13164909   0.009223   
2210  5930  삼성전자  20231227  76700  78000  76500  78000  20651042   0.018277   
2211  5930  삼성전자  20231228  77700  78500  77500  78500  17797536   0.006410   

      high_close_ratio  ...     ma20         macd  

In [35]:
import numpy as np
import pandas as pd

# 2019-02-13 이후의 데이터만 필터링

# format date type
df_samsung['date'] = df_samsung['date'].astype('str')
df_samsung['date'] = pd.to_datetime(df_samsung['date'])
df_skhynix['date'] = df_skhynix['date'].astype('str')
df_skhynix['date'] = pd.to_datetime(df_skhynix['date'])

df_samsung = df_samsung[df_samsung['date'] > '2020-01-01']

df_samsung_filter = df_samsung[['date', 'close', 'diffratio']].rename(columns={'close': 'samsung_close', 'diffratio': 'samsung_diffratio'})
df_skhynix_filter = df_skhynix[['date', 'close', 'diffratio']].rename(columns={'close': 'skhynix_close', 'diffratio': 'skhynix_diffratio'})

# merge two stocks
df = pd.merge(df_samsung_filter, df_skhynix_filter, on='date', how='left')

df.set_index('date', inplace=True)


In [9]:
df

,samsung_close,samsung_diffratio,skhynix_close,skhynix_diffratio
date,,,,
2020-01-02,55200,-0.010753,94700,0.006376
2020-01-03,55500,0.005435,94500,-0.002112
2020-01-06,55500,0.000000,94300,-0.002116
2020-01-07,55800,0.005405,94000,-0.003181
2020-01-08,56800,0.017921,97400,0.036170
...,...,...,...,...
2023-12-21,75000,0.002674,140500,0.000000
2023-12-22,75900,0.012000,140600,0.000712
2023-12-26,76600,0.009223,140900,0.002134


In [31]:

# 주식 티커 및 거래 예산 설정
tickers = ['samsung', 'skhynix']
budget = 100000000  # 거래 예산 100,000,000원

# 거래 수수료 및 거래세 설정
transaction_fee_rate = 0.00015  # 0.015%
transaction_tax_rate = 0.002     # 0.2%

# 종가 데이터 가져오기
close_prices = df[[f'{ticker}_close' for ticker in tickers]].iloc[0]

# 최대 거래 가능한 주식 수 계산
max_shares = {}
for ticker in tickers:
    max_shares[ticker] = budget * 0.5 // close_prices[f'{ticker}_close']  # 소수점 없이 최대 주식 수 계산

# 각 주식의 수익률 계산
daily_returns = df[[f'{ticker}_close' for ticker in tickers]].pct_change().dropna()

# 포트폴리오 수익률 계산 (동일 비율로 투자)
weights = np.array([max_shares['samsung'], max_shares['skhynix']])  # 각 주식의 최대 주식 수
total_investment = sum(weights * close_prices)

# 거래 수수료 및 거래세 계산
total_transaction_fee = total_investment * transaction_fee_rate
total_transaction_tax = total_investment * transaction_tax_rate
net_investment = total_investment - (total_transaction_fee + total_transaction_tax)

# 포트폴리오 수익률 계산 (순 투자를 기준으로)
portfolio_returns = (daily_returns.dot(weights)) / net_investment

# 샤프 지수 계산
risk_free_rate = 0.03 / 252  # 연간 무위험 수익률을 일일로 변환 (예: 3%)
mean_return = portfolio_returns.mean()
std_dev = portfolio_returns.std()

# 샤프 지수 계산
sharpe_ratio = (mean_return - risk_free_rate) / std_dev

# 결과 출력
print(f"{tickers[0]}의 최대 거래 가능한 주식 수: {max_shares['samsung']}주")
print(f"{tickers[1]}의 최대 거래 가능한 주식 수: {max_shares['skhynix']}주")
print(f"포트폴리오의 샤프 지수: {sharpe_ratio:.4f}")

print(f"포트폴리오 PV: {max_shares['samsung'] * 78500 + max_shares['skhynix'] * 141500}")

samsung의 최대 거래 가능한 주식 수: 636.0주
skhynix의 최대 거래 가능한 주식 수: 353.0주
포트폴리오의 샤프 지수: -703.8817
포트폴리오 PV: 99875500.0


In [41]:
import numpy as np
import pandas as pd

# 데이터프레임 예시 (실제 데이터로 교체해야 합니다)

# df.set_index('date', inplace=True)

# 주식 티커 및 거래 예산 설정
tickers = ['samsung', 'skhynix']
budget = 100000000  # 거래 예산 100,000,000원

# 거래 수수료 및 거래세 설정
transaction_fee_rate = 0.00015  # 0.015%
transaction_tax_rate = 0.002     # 0.2%

# 종가 데이터 가져오기
close_prices = df[[f'{ticker}_close' for ticker in tickers]].iloc[0]

# 최대 거래 가능한 주식 수 계산
max_shares = {}
for ticker in tickers:
    max_shares[ticker] = budget * 0.5 // close_prices[f'{ticker}_close']  # 소수점 없이 최대 주식 수 계산

# 각 주식의 수익률 계산
daily_returns = df[[f'{ticker}_close' for ticker in tickers]].pct_change().dropna()

# 포트폴리오 투자 금액 계산
investment_per_stock = budget * 0.5  # 각 주식에 동일 비율로 투자
total_investment = 0
for ticker in tickers:
    shares = max_shares[ticker]
    investment = shares * close_prices[f'{ticker}_close']
    total_investment += investment

# 거래 수수료 및 거래세 계산
total_transaction_fee = total_investment * transaction_fee_rate
total_transaction_tax = total_investment * transaction_tax_rate
net_investment = total_investment - (total_transaction_fee + total_transaction_tax)

# 포트폴리오 수익률 계산 (순 투자를 기준으로)
weights = np.array([max_shares['samsung'], max_shares['skhynix']])  # 각 주식의 최대 주식 수
portfolio_returns = (daily_returns.dot(weights)) / net_investment

# 샤프 지수 계산 
risk_free_rate = 0.03 / 252  # 연간 무위험 수익률을 일일로 변환 (예: 3%)
mean_return = portfolio_returns.mean()
std_dev = portfolio_returns.std()

# 샤프 지수 계산
sharpe_ratio = (mean_return - risk_free_rate) / std_dev

# 결과 출력
print(f"{tickers[0]}의 최대 거래 가능한 주식 수: {max_shares['samsung']}주")
print(f"{tickers[1]}의 최대 거래 가능한 주식 수: {max_shares['skhynix']}주")
print(f"포트폴리오의 샤프 지수: {sharpe_ratio:.4f}")

print(f"포트폴리오 PV: {max_shares['samsung'] * 78500 + max_shares['skhynix'] * 141500}")

samsung의 최대 거래 가능한 주식 수: 905.0주
skhynix의 최대 거래 가능한 주식 수: 527.0주
포트폴리오의 샤프 지수: -484.1920
포트폴리오 PV: 145613000.0
